In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat


def main():
    train_x = pd.read_csv("\\input\\train_test\\cdk2_train.csv",dtype=np.float32)
    test_x = pd.read_csv("train_test\\cdk2_test.csv",dtype=np.float32)
    train_y = pd.read_csv("G:\\projects\\cancer inhibitors\\train_test\\cdk2_trainlabels.csv",dtype=np.float32)
    test_y = pd.read_csv("G:\\projects\\cancer inhibitors\\train_test\\cdk_2testlabels.csv",dtype=np.float32)


    train_x=train_x.values
    test_x=test_x.values
    train_y=train_y.values
    test_y=test_y.values
   
    
    nb_classes = 2
    targets = train_y.reshape(-1)
    targets=targets.astype(int,casting='unsafe')
    train_y = np.eye(nb_classes)[targets]
    
    targets = test_y.reshape(-1)
    targets=targets.astype(int,casting='unsafe')
    test_y = np.eye(nb_classes)[targets]
    
    
   
    # Layer's sizes
    x_size = train_x.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 200               # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float32", shape=[None, x_size])
    y = tf.placeholder("float32", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(100):
        # Train with each example
        for i in range(len(train_x)):
            sess.run(updates, feed_dict={X: train_x[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_x, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_x, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()
if __name__ == '__main__':
    main()